In [71]:
import os
import cv2
from ultralytics import YOLO
from pathlib import Path
import numpy as np

In [72]:
# -----------------------
# Load trained YOLO model
# -----------------------
model = YOLO("../runs/segment/rice_yolo_runs/yolov8m_seg/weights/best.pt")

# source_dir = "../raw_dataset/1509"
# source_dir = "../raw_dataset/IRRI-6"
source_dir = "../raw_dataset/SUPER WHITE"

# save_root = "grain_crops"

img_size = 224  # CNN input size

# os.makedirs(save_root, exist_ok=True)

In [73]:
save_root = "../rice_dataset_02"

os.makedirs(save_root, exist_ok=True)

target_class_name = "SUPER WHITE"

# create only one folder
class_dir = os.path.join(save_root, target_class_name)
os.makedirs(class_dir, exist_ok=True)


In [74]:
# =====================================================
# PAD + RESIZE FUNCTION (BEST QUALITY)
# =====================================================

def pad_and_resize(img, size=224):
    """
    Keep aspect ratio + sharp resize
    (prevents distortion and blur)
    """
    h, w = img.shape[:2]

    # create square canvas
    m = max(h, w)
    canvas = np.zeros((m, m, 3), dtype=np.uint8)

    y_offset = (m - h) // 2
    x_offset = (m - w) // 2

    canvas[y_offset:y_offset+h, x_offset:x_offset+w] = img

    # high-quality resize
    canvas = cv2.resize(canvas, (size, size), interpolation=cv2.INTER_LANCZOS4)

    return canvas


In [75]:
# # -----------------------
# # Predict (ONLY boxes)
# # -----------------------
# results = model.predict(
#     source=source_dir,
#     conf=0.25,
#     stream=True
# )

# counter = 0

# for r in results:

#     img = r.orig_img
#     boxes = r.boxes.xyxy.cpu().numpy()

#     for box in boxes:   # ← no cls anymore

#         x1, y1, x2, y2 = map(int, box)

#         crop = img[y1:y2, x1:x2]
#         crop = cv2.resize(crop, (img_size, img_size))

#         save_path = os.path.join(class_dir, f"{target_class_name}_{counter}.jpg")

#         cv2.imwrite(save_path, crop)

#         counter += 1

# print("✅ Cropping finished (manual class folder used)")


In [76]:


# =====================================================
# PREDICT + CROP
# =====================================================

print("🚀 Cropping grains...")

results = model.predict(
    source=source_dir,
    conf=0.25,
    stream=True,
    save=False,
    verbose=False
)

counter = 0

for r in results:

    img = r.orig_img  # original image (NO drawings)
    boxes = r.boxes.xyxy.cpu().numpy()

    for box in boxes:

        x1, y1, x2, y2 = map(int, box)

        crop = img[y1:y2, x1:x2]

        if crop.size == 0:
            continue

        crop = pad_and_resize(crop, img_size)

        save_path = os.path.join(class_dir, f"{target_class_name}_{counter}.jpg")

        cv2.imwrite(save_path, crop)

        counter += 1


print(f"\n✅ Done! {counter} crops saved to:\n{class_dir}")

🚀 Cropping grains...

✅ Done! 8730 crops saved to:
../rice_dataset_02\SUPER WHITE
